[View in Colaboratory](https://colab.research.google.com/github/brucecmd/learn_gluon/blob/master/gluon_weight_decay1.ipynb)

In [0]:
from mxnet import nd, autograd, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn
from mxnet import init

In [0]:
num_train = 20
num_test = 100
num_inputs = 200
batch_size = 1
x = nd.random_normal(0,1,shape=(num_train+num_test, num_inputs))
true_w = nd.ones(shape=(num_inputs, 1))
true_b = 2.8
y = nd.dot(x,true_w) + true_b
y += nd.random_normal(0,0.01,shape=y.shape)

feature_train, feature_test = x[:num_train], x[num_train:]
label_train, label_test = y[:num_train], y[num_train:]
train_iter = gdata.DataLoader(gdata.ArrayDataset(feature_train, label_train), batch_size, shuffle=True)
test_iter = gdata.DataLoader(gdata.ArrayDataset(feature_test, label_test), batch_size, shuffle=True)

In [0]:
net = nn.Sequential()
net.add(nn.Flatten())
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))

In [0]:
loss_func = gloss.L2Loss()

In [0]:
trainer_w = gluon.Trainer(net.collect_params('.*weight'), 'sgd', {'learning_rate':0.001})
trainer_b = gluon.Trainer(net.collect_params('.*bias'), 'sgd', {'learning_rate':0.001})

In [233]:
epochs = 10
lr = 0.001
batch_size = 1
for i in range(epochs):
    for data, label in train_iter:
        with autograd.record():
            y_hat = net(data)
            l = loss_func(y_hat, label)
        l.backward()
        trainer_w.step(batch_size)
        trainer_b.step(batch_size)
    train_loss = loss_func(net(feature_train),label_train).mean().asscalar()
    test_loss = loss_func(net(feature_test),label_test).mean().asscalar()
    print('epoch[%d], train loss[%f], test loss[%f]'%(i, train_loss, test_loss))

epoch[0], train loss[49.454857], test loss[102.331932]
epoch[1], train loss[30.437103], test loss[101.611618]
epoch[2], train loss[19.049978], test loss[101.221725]
epoch[3], train loss[12.103906], test loss[101.032005]
epoch[4], train loss[7.799559], test loss[100.912109]
epoch[5], train loss[5.105788], test loss[100.871300]
epoch[6], train loss[3.379375], test loss[100.857765]
epoch[7], train loss[2.261538], test loss[100.868553]
epoch[8], train loss[1.526597], test loss[100.892952]
epoch[9], train loss[1.042768], test loss[100.912888]
